In [1]:
import pandas as pd

In [2]:
train_rides_df = pd.read_csv('DBtrainrides.csv')

In [3]:
train_rides_df.head()

,ID,line,path,eva_nr,category,station,state,city,zip,long,lat,arrival_plan,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info,arrival_delay_check,departure_delay_check
0,1573967790757085557-2407072312-14,20,Stolberg(Rheinl)Hbf Gl.44|Eschweiler-St.Jöris|...,8000001,2,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.767800,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time
1,349781417030375472-2407080017-1,18,NaN,8000001,2,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.767800,NaN,2024-07-08 00:17:00,NaN,NaN,0,0,NaN,on_time,on_time
2,7157250219775883918-2407072120-25,1,Hamm(Westf)Hbf|Kamen|Kamen-Methler|Dortmund-Ku...,8000406,4,Aachen-Rothe Erde,Nordrhein-Westfalen,Aachen,52066,6.116475,50.770202,2024-07-08 00:03:00,2024-07-08 00:04:00,2024-07-08 00:03:00,2024-07-08 00:04:00,0,0,NaN,on_time,on_time
3,349781417030375472-2407080017-2,18,Aachen Hbf,8000404,5,Aachen West,Nordrhein-Westfalen,Aachen,52072,6.070715,50.780360,2024-07-08 00:20:00,2024-07-08 00:21:00,NaN,NaN,0,0,NaN,on_time,on_time
4,1983158592123451570-2407080010-3,33,Herzogenrath|Kohlscheid,8000404,5,Aachen West,Nordrhein-Westfalen,Aachen,52072,6.070715,50.780360,2024-07-08 00:20:00,2024-07-08 00:21:00,2024-07-08 00:20:00,2024-07-08 00:21:00,0,0,NaN,on_time,on_time


In [4]:
zip_df=train_rides_df.groupby("zip").size().reset_index(name="count")
zip_df.head()

,zip,count
0,1067,2458
1,1069,2045
2,1097,3305
3,1109,1800
4,1127,597


In [5]:
train_rides_df.count()

ID                       2061357
line                     2061357
path                     1850002
eva_nr                   2061357
category                 2061357
station                  2061357
state                    2061357
city                     2061357
zip                      2061357
long                     2061357
lat                      2061357
arrival_plan             1850002
departure_plan           2061357
arrival_change           1585727
departure_change         1721431
arrival_delay_m          2061357
departure_delay_m        2061357
info                      645341
arrival_delay_check      2061357
departure_delay_check    2061357
dtype: int64

2,061,357–1,850,002 = 211,355 fewer cases in the 'path' column

In [6]:
# Step 2: Create a new DataFrame with a count of how often each 'eva_nr' is mentioned
station_mapping_df = train_rides_df.groupby(['eva_nr', 'station', 'category', 'city', 'long', 'lat']).size().reset_index(name='count')

# Step 3: Optionally, reset the index for a cleaner DataFrame
station_mapping_df = station_mapping_df.reset_index(drop=True)

# Display the new DataFrame
station_mapping_df.head()

,eva_nr,station,category,city,long,lat,count
0,8000001,Aachen Hbf,2,Aachen,6.091499,50.767800,1488
1,8000002,Aalen Hbf,3,Aalen,10.096271,48.841013,823
2,8000004,Altenbeken,3,Altenbeken,8.943319,51.766433,848
3,8000007,Alzey,3,Alzey,8.109749,49.750200,591
4,8000009,Ansbach,3,Ansbach,10.578239,49.298032,829


In [7]:
station_mapping_df.count()

eva_nr      1996
station     1996
category    1996
city        1996
long        1996
lat         1996
count       1996
dtype: int64

what is the average number of stations per category? go check the 'category' column

In [8]:
# # Step 1: Split the 'path' into individual stations where 'path' is not empty
# # Create a new DataFrame that splits the 'path' column
# stations_df = train_rides_df['path'].str.split('|', expand=True)
# 
# # Step 2: Keep the same 'eva_nr', 'station', and 'city' columns, add the split station columns
# split_stations_df = pd.concat([train_rides_df[['ID','eva_nr', 'station', 'city']], stations_df], axis=1)
# 
# split_stations_df.head()

In [9]:
# Step 1: Split the 'path' into individual stations where 'path' is not empty
train_rides_df['path'] = train_rides_df['path'].fillna('')

# Step 2: Split each 'path' entry by '|', creating multiple rows per station
split_stations_df = train_rides_df.assign(current_station=train_rides_df['path'].str.split('|')).explode('current_station').drop(columns=['path'])

# Display the result
split_stations_df.head()

,ID,line,eva_nr,category,station,state,city,zip,long,lat,arrival_plan,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info,arrival_delay_check,departure_delay_check,current_station
0,1573967790757085557-2407072312-14,20,8000001,2,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.7678,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Stolberg(Rheinl)Hbf Gl.44
0,1573967790757085557-2407072312-14,20,8000001,2,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.7678,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Eschweiler-St.Jöris
0,1573967790757085557-2407072312-14,20,8000001,2,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.7678,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf Poststraße
0,1573967790757085557-2407072312-14,20,8000001,2,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.7678,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf-Mariadorf
0,1573967790757085557-2407072312-14,20,8000001,2,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.7678,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf-Kellersberg


In [10]:
split_stations_df.drop(['category', 'station', 'state', 'city', 'zip', 'long', 'lat'], axis=1, inplace=True)
split_stations_df.head()

,ID,line,eva_nr,arrival_plan,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info,arrival_delay_check,departure_delay_check,current_station
0,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Stolberg(Rheinl)Hbf Gl.44
0,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Eschweiler-St.Jöris
0,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf Poststraße
0,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf-Mariadorf
0,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf-Kellersberg


In [11]:
# Perform the merge operation
# First, ensure the station names match by stripping any extra whitespace
station_mapping_df['station'] = station_mapping_df['station'].str.strip()
split_stations_df['current_station'] = split_stations_df['current_station'].str.strip()
split_stations_df_copy= split_stations_df

# Merge the DataFrames on the station name (current_station and station)
station_internal_eva_mapping_df = split_stations_df.merge(station_mapping_df, how='left', left_on='current_station', right_on='station')

In [12]:
station_internal_eva_mapping_df

,ID,line,eva_nr_x,arrival_plan,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info,arrival_delay_check,departure_delay_check,current_station,eva_nr_y,station,category,city,long,lat,count
0,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Stolberg(Rheinl)Hbf Gl.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Eschweiler-St.Jöris,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf Poststraße,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf-Mariadorf,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf-Kellersberg,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19714102,3370285438001482281-2407142234-7,8,8003775,2024-07-14 23:10:00,2024-07-14 23:11:00,2024-07-14 23:11:00,2024-07-14 23:12:00,1,1,Information,on_time,on_time,Lübeck Hbf,8000237.0,Lübeck Hbf,2.0,Lübeck,10.669821,53.867547,1428.0
19714103,-8774053210575864323-2407142305-3,80,8003775,2024-07-14 23:17:00,2024-07-14 23:18:00,2024-07-14 23:17:00,2024-07-14 23:18:00,0,0,Information,on_time,on_time,Bad Oldesloe,8000023.0,Bad Oldesloe,2.0,Bad Oldesloe,10.382541,53.805390,800.0
19714104,-8774053210575864323-2407142305-3,80,8003775,2024-07-14 23:17:00,2024-07-14 23:18:00,2024-07-14 23:17:00,2024-07-14 23:18:00,0,0,Information,on_time,on_time,Reinfeld(Holst),NaN,NaN,NaN,NaN,NaN,NaN,NaN
19714105,-1537118689903044118-2407142354-1,11,8001580,NaN,2024-07-14 23:54:00,NaN,NaN,0,0,Information. (Quelle: zuginfo.nrw),on_time,on_time,,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
station_mapping_df

,eva_nr,station,category,city,long,lat,count
0,8000001,Aachen Hbf,2,Aachen,6.091499,50.767800,1488
1,8000002,Aalen Hbf,3,Aalen,10.096271,48.841013,823
2,8000004,Altenbeken,3,Altenbeken,8.943319,51.766433,848
3,8000007,Alzey,3,Alzey,8.109749,49.750200,591
4,8000009,Ansbach,3,Ansbach,10.578239,49.298032,829
...,...,...,...,...,...,...,...
1991,8089472,Lichterfelde Süd,5,Berlin,13.309040,52.410899,1544
1992,8089473,Osdorferstraße,5,Berlin,13.314288,52.418841,1536
1993,8089474,Schöneberg,4,Berlin,13.352068,52.479377,5831
1994,8089537,Julius-Leber-Brücke,5,Berlin,13.361266,52.486519,2191


Visualise the stations in a map (/grid) (without it being mapped in a real map) and see if there are any patterns. George \\

Additionally, use the same map to see the delay (in % of total rides) per station and per zip. Delay in this case would be if the train did not arrive / leave at the intended time. George \\


See if we can use K-means to cluster the stations into mini regions to see if the entire region has delay problems. We do not know what k to use, so we can try a few values. Check the distance per stations and see by eye if the clusters make sense to create. Raphael

In [14]:
station_internal_eva_mapping_df.head(10000)

,ID,line,eva_nr_x,arrival_plan,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info,arrival_delay_check,departure_delay_check,current_station,eva_nr_y,station,category,city,long,lat,count
0,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Stolberg(Rheinl)Hbf Gl.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Eschweiler-St.Jöris,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf Poststraße,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf-Mariadorf,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Alsdorf-Kellersberg,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-8894360290589207845-2407080011-17,4,8001229,2024-07-08 00:49:00,2024-07-08 00:51:00,2024-07-08 00:51:00,2024-07-08 00:51:00,2,0,NaN,on_time,on_time,München-Aubing,8004141.0,München-Aubing,5.0,München,11.413309,48.155968,871.0
9996,-8894360290589207845-2407080011-17,4,8001229,2024-07-08 00:49:00,2024-07-08 00:51:00,2024-07-08 00:51:00,2024-07-08 00:51:00,2,0,NaN,on_time,on_time,Puchheim,8004893.0,Puchheim,4.0,Puchheim,11.353061,48.172179,896.0
9997,-8894360290589207845-2407080011-17,4,8001229,2024-07-08 00:49:00,2024-07-08 00:51:00,2024-07-08 00:51:00,2024-07-08 00:51:00,2,0,NaN,on_time,on_time,Eichenau(Oberbay),NaN,NaN,NaN,NaN,NaN,NaN,NaN
9998,-8894360290589207845-2407080011-17,4,8001229,2024-07-08 00:49:00,2024-07-08 00:51:00,2024-07-08 00:51:00,2024-07-08 00:51:00,2,0,NaN,on_time,on_time,Fürstenfeldbruck,8002141.0,Fürstenfeldbruck,4.0,Fürstenfeldbruck,11.262148,48.171656,1068.0


In [45]:
eva_df = pd.read_csv('stations.csv')
eva_df

,Station Name,IBNR
0,Aachen Hbf,8000001
1,Aachen Schanz,8070704
2,Aachen Süd(Gr),8000403
3,Aachen West,8000404
4,Aachen-Rothe Erde,8000406
...,...,...
7662,Zwingenberg(Bergstr),8006687
7663,Zwönitz,8013435
7664,Zwota,8013437
7665,Zwota-Zechenbach,8013436


In [46]:
# Entferne mögliche führende oder nachgestellte Leerzeichen und setze auf Kleinbuchstaben
split_stations_df['current_station'] = split_stations_df['current_station'].str.strip().str.lower()
eva_df['Station Name'] = eva_df['Station Name'].str.strip().str.lower()

# Updated merge using 'station' instead of 'Station Name'
station_external_eva_mapping_df = split_stations_df.merge(
    eva_df, 
    how='left', 
    left_on='current_station', 
    right_on='Station Name'
)

In [47]:
station_external_eva_mapping_df

,ID,line,eva_nr,arrival_plan,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info,arrival_delay_check,departure_delay_check,current_station,Station Name,IBNR
0,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,stolberg(rheinl)hbf gl.44,NaN,NaN
1,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,eschweiler-st.jöris,eschweiler-st.jöris,8001917.0
2,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,alsdorf poststraße,alsdorf poststraße,8000510.0
3,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,alsdorf-mariadorf,alsdorf-mariadorf,8000527.0
4,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,alsdorf-kellersberg,alsdorf-kellersberg,8000521.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19785712,3370285438001482281-2407142234-7,8,8003775,2024-07-14 23:10:00,2024-07-14 23:11:00,2024-07-14 23:11:00,2024-07-14 23:12:00,1,1,Information,on_time,on_time,lübeck hbf,lübeck hbf,8000237.0
19785713,-8774053210575864323-2407142305-3,80,8003775,2024-07-14 23:17:00,2024-07-14 23:18:00,2024-07-14 23:17:00,2024-07-14 23:18:00,0,0,Information,on_time,on_time,bad oldesloe,bad oldesloe,8000023.0
19785714,-8774053210575864323-2407142305-3,80,8003775,2024-07-14 23:17:00,2024-07-14 23:18:00,2024-07-14 23:17:00,2024-07-14 23:18:00,0,0,Information,on_time,on_time,reinfeld(holst),reinfeld(holst),8005019.0
19785715,-1537118689903044118-2407142354-1,11,8001580,NaN,2024-07-14 23:54:00,NaN,NaN,0,0,Information. (Quelle: zuginfo.nrw),on_time,on_time,,NaN,NaN


In [48]:
# Count where 'Station Name' is NaN and 'current_station' is not NaN
count_nan_station_name = station_external_eva_mapping_df[
    (station_external_eva_mapping_df['Station Name'].isna()) & 
    (station_external_eva_mapping_df['current_station'].notna())
].shape[0]
count_nan_station_name

1310276

In [28]:
station_external_eva_mapping_df.columns

Index(['ID', 'line', 'eva_nr', 'arrival_plan', 'departure_plan',
       'arrival_change', 'departure_change', 'arrival_delay_m',
       'departure_delay_m', 'info', 'arrival_delay_check',
       'departure_delay_check', 'current_station', 'Station Name', 'IBNR'],
      dtype='object')

In [29]:
# Convert empty strings back to NaN in 'path' column
station_external_eva_mapping_df['current_station'] = station_external_eva_mapping_df['current_station'].replace('', pd.NA)

# Count rows where 'station' is NaN
nan_station_count = station_external_eva_mapping_df['current_station'].isna().sum()
print(f"Number of rows with NaN station names: {nan_station_count}")

Number of rows with NaN station names: 211355


In [30]:
# Count rows where 'station' is NaN
nan_station_count = station_external_eva_mapping_df['Station Name'].isna().sum()
print(f"Number of rows with NaN station names: {nan_station_count}")

Number of rows with NaN station names: 1313277


In [33]:
filtered_df = station_external_eva_mapping_df[station_external_eva_mapping_df['Station Name'].isna()]

In [34]:
filtered_df

,ID,line,eva_nr,arrival_plan,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info,arrival_delay_check,departure_delay_check,current_station,Station Name,IBNR
0,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Stolberg(Rheinl)Hbf Gl.44,NaN,NaN
13,349781417030375472-2407080017-1,18,8000001,NaN,2024-07-08 00:17:00,NaN,NaN,0,0,NaN,on_time,on_time,<NA>,NaN,NaN
14,7157250219775883918-2407072120-25,1,8000406,2024-07-08 00:03:00,2024-07-08 00:04:00,2024-07-08 00:03:00,2024-07-08 00:04:00,0,0,NaN,on_time,on_time,Hamm(Westf)Hbf,NaN,NaN
47,-2100556839975301087-2407072307-13,18,8000404,2024-07-08 00:37:00,2024-07-08 00:41:00,2024-07-08 00:37:00,2024-07-08 00:41:00,0,0,NaN,on_time,on_time,Liège-Guillemins,NaN,NaN
48,-2100556839975301087-2407072307-13,18,8000404,2024-07-08 00:37:00,2024-07-08 00:41:00,2024-07-08 00:37:00,2024-07-08 00:41:00,0,0,NaN,on_time,on_time,Bressoux,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19785699,-4498532330426324655-2407142201-14,RE18,8003105,2024-07-14 23:55:00,2024-07-14 23:56:00,2024-07-14 23:55:00,2024-07-14 23:56:00,0,0,Bauarbeiten,on_time,on_time,Oldenburg(Oldb)Hbf,NaN,NaN
19785701,-5558360799253050120-2407142310-4,RE18,8003105,2024-07-14 23:33:00,2024-07-14 23:33:00,2024-07-14 23:33:00,2024-07-14 23:33:00,0,0,Bauarbeiten,on_time,on_time,Wilhelmshaven,NaN,NaN
19785711,3370285438001482281-2407142234-7,8,8003775,2024-07-14 23:10:00,2024-07-14 23:11:00,2024-07-14 23:11:00,2024-07-14 23:12:00,1,1,Information,on_time,on_time,Lübeck-Dänischburg IKEA,NaN,NaN
19785715,-1537118689903044118-2407142354-1,11,8001580,NaN,2024-07-14 23:54:00,NaN,NaN,0,0,Information. (Quelle: zuginfo.nrw),on_time,on_time,<NA>,NaN,NaN


In [35]:
eva_v2_df = pd.read_csv('stations_v2.csv')
eva_v2_df

,IBNR,Station Name
0,8000001,Aachen Hbf
1,8000001,AC
2,8000001,Aken C
3,8000001,Aquisgrana
4,8000001,Aix-la-Chapelle
...,...,...
9128,8098553,Hamburg-Altona(S)
9129,8098555,Bensersiel NE
9130,8099503,Hildesheim Gbf
9131,8099506,Stolberg(Rheinl)Gbf


In [36]:
# Updated merge using 'station' instead of 'Station Name'
station_external_eva_mapping_v2_df = split_stations_df.merge(
    eva_v2_df, 
    how='left', 
    left_on='current_station', 
    right_on='Station Name'
)

In [40]:
# Convert empty strings in the 'current_station' column to NaN values
station_external_eva_mapping_v2_df['current_station'] = station_external_eva_mapping_v2_df['current_station'].replace('', pd.NA)

# Count and display the number of rows with NaN values in the 'current_station' column
nan_current_station_count = station_external_eva_mapping_v2_df['current_station'].isna().sum()
print(f"Number of rows with NaN values in 'current_station': {nan_current_station_count}")

# Count and display the number of rows with NaN values in the 'Station Name' column
nan_station_name_count = station_external_eva_mapping_v2_df['Station Name'].isna().sum()
print(f"Number of rows with NaN values in 'Station Name': {nan_station_name_count}")

Number of rows with NaN values in 'current_station': 211355
Number of rows with NaN values in 'Station Name': 1129571


In [42]:
filtered_v2_df = station_external_eva_mapping_df[station_external_eva_mapping_df['Station Name'].isna() & station_external_eva_mapping_df['current_station'].notna()]
filtered_v2_df

,ID,line,eva_nr,arrival_plan,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info,arrival_delay_check,departure_delay_check,current_station,Station Name,IBNR
0,1573967790757085557-2407072312-14,20,8000001,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time,Stolberg(Rheinl)Hbf Gl.44,NaN,NaN
14,7157250219775883918-2407072120-25,1,8000406,2024-07-08 00:03:00,2024-07-08 00:04:00,2024-07-08 00:03:00,2024-07-08 00:04:00,0,0,NaN,on_time,on_time,Hamm(Westf)Hbf,NaN,NaN
47,-2100556839975301087-2407072307-13,18,8000404,2024-07-08 00:37:00,2024-07-08 00:41:00,2024-07-08 00:37:00,2024-07-08 00:41:00,0,0,NaN,on_time,on_time,Liège-Guillemins,NaN,NaN
48,-2100556839975301087-2407072307-13,18,8000404,2024-07-08 00:37:00,2024-07-08 00:41:00,2024-07-08 00:37:00,2024-07-08 00:41:00,0,0,NaN,on_time,on_time,Bressoux,NaN,NaN
49,-2100556839975301087-2407072307-13,18,8000404,2024-07-08 00:37:00,2024-07-08 00:41:00,2024-07-08 00:37:00,2024-07-08 00:41:00,0,0,NaN,on_time,on_time,Vise,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19785381,5391302266099657590-2407142320-8,68,8006283,2024-07-14 23:50:00,2024-07-14 23:50:00,2024-07-14 23:50:00,2024-07-14 23:51:00,0,1,Information,on_time,on_time,Neu-Edingen/Friedrichsfeld,NaN,NaN
19785385,5391302266099657590-2407142320-8,68,8006283,2024-07-14 23:50:00,2024-07-14 23:50:00,2024-07-14 23:50:00,2024-07-14 23:51:00,0,1,Information,on_time,on_time,Weinheim(Bergstr)Hbf,NaN,NaN
19785699,-4498532330426324655-2407142201-14,RE18,8003105,2024-07-14 23:55:00,2024-07-14 23:56:00,2024-07-14 23:55:00,2024-07-14 23:56:00,0,0,Bauarbeiten,on_time,on_time,Oldenburg(Oldb)Hbf,NaN,NaN
19785701,-5558360799253050120-2407142310-4,RE18,8003105,2024-07-14 23:33:00,2024-07-14 23:33:00,2024-07-14 23:33:00,2024-07-14 23:33:00,0,0,Bauarbeiten,on_time,on_time,Wilhelmshaven,NaN,NaN


In [20]:
import plotly.express as px

In [21]:
# # Generate a blank point graph
# fig_blank = px.scatter(train_rides_df, x="long", y="lat", title="Blank Point Graph of Longitude and Latitude")
# 
# # Show plot
# fig_blank.show()

In [22]:
# # Generate a map using longitude and latitude
# fig_map = px.scatter_mapbox(train_rides_df, lat="lat", lon="long", 
#                             mapbox_style="open-street-map", 
#                             title="Map of Train Stations Based on Longitude and Latitude")
# # Show plot
# fig_map.show()